## Limpeza de dados Cartola ano 2017

In [161]:
from pyspark.sql import HiveContext
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lower, col, lit, regexp_replace
from pyspark.sql.functions import lit
import pyspark.sql.functions as f
import pandas as pd
import json
import requests

## Dados Agregados (Ainda não consegui interpretar)

In [23]:
#Carrega arquivo CSV
dados_agregados_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_dados_agregados.csv", header=True)
dados_agregados_2017_df.toPandas()

,AtletaID,Rodada,ClubeID,Participou,Posicao,Jogos,Pontos,PontosMedia,Preco,PrecoVariacao,...,risk_points,mes,dia,home.score.x,away.score.x,pred.home.score,pred.away.score,home.attack,home.defend,variable
0,36443,1,285,TRUE,gol,1,5,5,10.6,1.6,...,1,04,19,1,0,NA,NA,NA,NA,home.team
1,36443,2,285,TRUE,gol,2,-3,1,8.27,-2.33,...,1,04,27,2,2,NA,NA,NA,NA,away.team
2,36443,3,285,TRUE,gol,3,-2.6,-0.2,6.81,-1.46,...,1,05,04,2,1,NA,NA,NA,NA,home.team
3,36443,4,285,TRUE,gol,4,4,0.85,7.96,1.15,...,1,05,10,2,1,NA,NA,NA,NA,home.team
4,36443,5,285,TRUE,gol,5,5,1.68,8.61,0.65,...,1,05,18,0,0,NA,NA,NA,NA,away.team
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43816,100800,38,277,TRUE,lat,1,-2.8,-2.8,0.79,-0.21,...,NA,12,03,1,1,1.21293936673161,0.749835089294531,0.409933658127241,-0.479318868720867,home.team
43817,100839,38,284,TRUE,zag,1,-0.8,-0.8,0.79,-0.21,...,NA,12,03,4,3,1.39611377257526,1.4615820244055,0.790100011440696,-0.113050657889775,away.team
43818,100840,38,284,TRUE,zag,1,-0.1,-0.1,0.79,-0.21,...,NA,12,03,4,3,1.39611377257526,1.4615820244055,0.790100011440696,-0.113050657889775,away.team
43819,100842,38,284,TRUE,zag,1,-0.1,-0.1,0.79,-0.21,...,NA,12,03,4,3,1.39611377257526,1.4615820244055,0.790100011440696,-0.113050657889775,away.team


In [26]:
# remove todas as linhas com rodada == 0
dados_agregados_2017_df = dados_agregados_2017_df[dados_agregados_2017_df['pred.home.score'] > 0]

AnalysisException: 'Cannot resolve column name "pred.home.score" among (AtletaID, Rodada, ClubeID, Participou, Posicao, Jogos, Pontos, PontosMedia, Preco, PrecoVariacao, FS, PE, A, FT, FD, FF, G, I, PP, RB, FC, GC, CA, CV, SG, DD, DP, GS, ano, Apelido, Status, avg.Points, avg.last05, avg.FS, avg.FS.l05, avg.PE, avg.PE.l05, avg.A, avg.A.l05, avg.FT, avg.FT.l05, avg.FD, avg.FD.l05, avg.FF, avg.FF.l05, avg.G, avg.G.l05, avg.I, avg.I.l05, avg.PP, avg.PP.l05, avg.RB, avg.RB.l05, avg.FC, avg.FC.l05, avg.GC, avg.GC.l05, avg.CA, avg.CV.l05, avg.SG, avg.SG.l05, avg.DD, avg.DD.l05, avg.DP, avg.DP.l05, avg.GS, avg.GS.l05, risk_points, mes, dia, home.score.x, away.score.x, pred.home.score, pred.away.score, home.attack, home.defend, variable);'

In [24]:
sorted_dados_agregados_2017_df = dados_agregados_2017_df.sort(dados_agregados_2017_df)

TypeError: Invalid argument, not a string or column: DataFrame[AtletaID: string, Rodada: string, ClubeID: string, Participou: string, Posicao: string, Jogos: string, Pontos: string, PontosMedia: string, Preco: string, PrecoVariacao: string, FS: string, PE: string, A: string, FT: string, FD: string, FF: string, G: string, I: string, PP: string, RB: string, FC: string, GC: string, CA: string, CV: string, SG: string, DD: string, DP: string, GS: string, ano: string, Apelido: string, Status: string, avg.Points: string, avg.last05: string, avg.FS: string, avg.FS.l05: string, avg.PE: string, avg.PE.l05: string, avg.A: string, avg.A.l05: string, avg.FT: string, avg.FT.l05: string, avg.FD: string, avg.FD.l05: string, avg.FF: string, avg.FF.l05: string, avg.G: string, avg.G.l05: string, avg.I: string, avg.I.l05: string, avg.PP: string, avg.PP.l05: string, avg.RB: string, avg.RB.l05: string, avg.FC: string, avg.FC.l05: string, avg.GC: string, avg.GC.l05: string, avg.CA: string, avg.CV.l05: string

In [25]:
#Adiciona coluna ANO = 2016
#Clube ID está como NA????
dados_agregados_2017_ano = sorted_dados_agregados_2017_df.withColumn('ano', lit(2017))
dados_agregados_2017_ano.show()

NameError: name 'sorted_dados_agregados_2017_df' is not defined

In [ ]:
dados_agregados_2017_ano.write.partitionBy('ano').parquet('/cartola/2017/clean/2017_dados_agregados')

In [ ]:
#transforma arquivo em parquet
dados_agregados_2017_parquet = spark.read.parquet('/cartola/2017/clean/2017_dados_agregados')

In [ ]:
dados_agregados_2017_parquet.show()
dados_agregados_2017_parquet.schema


In [ ]:
#Alterar ID para int.(está string)
jogadores_2016_parquet = jogadores_2016_parquet.withColumn("ID", jogadores_2016_parquet["ID"].cast(IntegerType()))
jogadores_2016_parquet = jogadores_2016_parquet.withColumn("ClubeID", jogadores_2016_parquet["ClubeID"].cast(IntegerType()))
jogadores_2016_parquet = jogadores_2016_parquet.withColumn("PosicaoID", jogadores_2016_parquet["PosicaoID"].cast(IntegerType()))

In [ ]:
jogadores_2016_parquet.schema

In [ ]:
jogadores_2016_parquet.show()

print(jogadores_2016_parquet.count())

## Jogadores

In [27]:
#Carrega arquivo CSV
jogadores_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_jogadores.csv", header=True)
sorted_jogadores_2017_df = jogadores_2017_df.sort(jogadores_2017_df.AtletaID.desc())


In [28]:
#Adiciona coluna ANO = 2017
#Clube ID está como NA????
jogadores_2017_ano = sorted_jogadores_2017_df.withColumn('ano', lit(2017))
jogadores_2017_ano.show()

+--------+----------------+-------+---------+----+
|AtletaID|         Apelido|ClubeID|PosicaoID| ano|
+--------+----------------+-------+---------+----+
|   99959|   Lucas Formiga|    303|        5|2017|
|   99918|         Patrick|    284|        4|2017|
|   99915|        Ezequiel|    263|        4|2017|
|   99903|    Lucas Campos|    263|        5|2017|
|   99900|    Luan Pereira|    314|        4|2017|
|   99894|  Junior Brumado|    265|        5|2017|
|   99892|        Luis Ali|    303|        5|2017|
|   99883|         Penilla|    315|        5|2017|
|   99882|  Fabiano Soares|    293|        6|2017|
|   99834|        Dionathã|    284|        5|2017|
|   99824|         Douglas|    265|        5|2017|
|   99818|        Paulinho|    267|        5|2017|
|   99817|          Bremer|    282|        3|2017|
|   99789|         Brenner|    276|        5|2017|
|   99721|     Mascarenhas|    266|        2|2017|
|   99670|          Arthur|    283|        3|2017|
|   99669|           Elano|    

In [31]:
jogadores_2017_ano.write.parquet('/cartola/clean/jogadores/2017')

In [32]:
#transforma arquivo em parquet
jogadores_2017_parquet = spark.read.parquet('/cartola/clean/jogadores/*')

In [33]:
jogadores_2017_parquet.show()
jogadores_2017_parquet.schema


+-----+--------------------+-------+---------+----+
|   ID|             Apelido|ClubeID|PosicaoID| ano|
+-----+--------------------+-------+---------+----+
|51683|        Bruno Rangel|    315|        5|2014|
|51705|       Bruno Rodrigo|    283|        3|2014|
|51772|     Éverton Ribeiro|    283|        4|2014|
|51779|       Pedro Botelho|    282|        2|2014|
|51781|               Ávine|    265|        2|2014|
|51985|      Anderson Pedra|    292|        4|2014|
|41387|       Eduardo Costa|    314|        4|2015|
|41327|Vanderlei Luxemburgo|    262|        6|2015|
|41258|     Adilson Batista|     NA|        6|2015|
|41218|              Magrão|    292|        1|2015|
|41159|         João Carlos|     NA|        3|2015|
|97460|         João Victor|    344|        4|2016|
|97451|      Talysson Lalau|    294|        4|2016|
|97450|    Gustavo Mosquito|    294|        5|2016|
|97449|            Carvalho|    294|        4|2016|
|97448|    Thalisson Kelven|    294|        3|2016|
|97445|   Ma

StructType(List(StructField(ID,StringType,true),StructField(Apelido,StringType,true),StructField(ClubeID,StringType,true),StructField(PosicaoID,StringType,true),StructField(ano,IntegerType,true)))

In [34]:
#Alterar ID para int.(está string)
jogadores_2017_parquet = jogadores_2017_parquet.withColumn("ID", jogadores_2017_parquet["ID"].cast(IntegerType()))
jogadores_2017_parquet = jogadores_2017_parquet.withColumn("ClubeID", jogadores_2017_parquet["ClubeID"].cast(IntegerType()))
jogadores_2017_parquet = jogadores_2017_parquet.withColumn("PosicaoID", jogadores_2017_parquet["PosicaoID"].cast(IntegerType()))

In [35]:
jogadores_2017_parquet.schema

StructType(List(StructField(ID,IntegerType,true),StructField(Apelido,StringType,true),StructField(ClubeID,IntegerType,true),StructField(PosicaoID,IntegerType,true),StructField(ano,IntegerType,true)))

In [36]:
jogadores_2017_parquet.show()

print(jogadores_2017_parquet.count())

+-----+--------------------+-------+---------+----+
|   ID|             Apelido|ClubeID|PosicaoID| ano|
+-----+--------------------+-------+---------+----+
|51683|        Bruno Rangel|    315|        5|2014|
|51705|       Bruno Rodrigo|    283|        3|2014|
|51772|     Éverton Ribeiro|    283|        4|2014|
|51779|       Pedro Botelho|    282|        2|2014|
|51781|               Ávine|    265|        2|2014|
|51985|      Anderson Pedra|    292|        4|2014|
|41387|       Eduardo Costa|    314|        4|2015|
|41327|Vanderlei Luxemburgo|    262|        6|2015|
|41258|     Adilson Batista|   null|        6|2015|
|41218|              Magrão|    292|        1|2015|
|41159|         João Carlos|   null|        3|2015|
|97460|         João Victor|    344|        4|2016|
|97451|      Talysson Lalau|    294|        4|2016|
|97450|    Gustavo Mosquito|    294|        5|2016|
|97449|            Carvalho|    294|        4|2016|
|97448|    Thalisson Kelven|    294|        3|2016|
|97445|   Ma

## Partidas_IDS (temos partidas como NA)

In [167]:
partidas_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_partidas.csv", header=True)

In [168]:
partidas_2017_df_ano = partidas_2017_df.withColumn('ano', lit(2017))

In [169]:
partidas_2017_df_ano.toPandas()

,_c0,game,round,date,home_team,score,away_team,arena,X,ano
0,1,1,1,14/05/2017 - 11:00,Fluminense - RJ,3 x 2,Santos - SP,Maracanã - Rio de Janeiro - RJ,None,2017
1,2,2,1,13/05/2017 - 16:00,Flamengo - RJ,1 x 1,Atlético - MG,Maracanã - Rio de Janeiro - RJ,None,2017
2,3,3,1,14/05/2017 - 16:00,Palmeiras - SP,4 x 0,Vasco da Gama - RJ,Allianz Parque - Sao Paulo - SP,None,2017
3,4,4,1,13/05/2017 - 19:00,Corinthians - SP,1 x 1,Chapecoense - SC,Arena Corinthians - Sao Paulo - SP,None,2017
4,5,5,1,14/05/2017 - 16:00,Cruzeiro - MG,1 x 0,São Paulo - SP,Mineirão - Belo Horizonte - MG,None,2017
...,...,...,...,...,...,...,...,...,...,...
375,376,376,38,03/12/2017 - 17:00,Atlético - PR,3 x 0,Palmeiras - SP,Arena da Baixada - Curitiba - PR,None,2017
376,377,377,38,03/12/2017 - 17:00,Sport - PE,1 x 0,Corinthians - SP,Ilha do Retiro - Recife - PE,None,2017
377,378,378,38,03/12/2017 - 17:00,Vitória - BA,1 x 2,Flamengo - RJ,Manoel Barradas - Salvador - BA,None,2017
378,379,379,38,03/12/2017 - 17:00,Atlético - GO,1 x 1,Fluminense - RJ,Olímpico Pedro Ludovico - Goiania - GO,None,2017


In [170]:
# Removendo colunas
partidas_2017_df_ano = partidas_2017_df_ano.drop('_c0', 'X', 'arena', 'date')

In [171]:
partidas_2017_df_ano.toPandas()

,game,round,home_team,score,away_team,ano
0,1,1,Fluminense - RJ,3 x 2,Santos - SP,2017
1,2,1,Flamengo - RJ,1 x 1,Atlético - MG,2017
2,3,1,Palmeiras - SP,4 x 0,Vasco da Gama - RJ,2017
3,4,1,Corinthians - SP,1 x 1,Chapecoense - SC,2017
4,5,1,Cruzeiro - MG,1 x 0,São Paulo - SP,2017
...,...,...,...,...,...,...
375,376,38,Atlético - PR,3 x 0,Palmeiras - SP,2017
376,377,38,Sport - PE,1 x 0,Corinthians - SP,2017
377,378,38,Vitória - BA,1 x 2,Flamengo - RJ,2017
378,379,38,Atlético - GO,1 x 1,Fluminense - RJ,2017


In [193]:
# Padronizando nome de colunas
partidas_2017_df_ano = partidas_2017_df_ano.withColumnRenamed("game", "ID")
partidas_2017_df_ano = partidas_2017_df_ano.withColumnRenamed("round", "Rodada")
partidas_2017_df_ano = partidas_2017_df_ano.withColumnRenamed("home_team", "CasaID")
partidas_2017_df_ano = partidas_2017_df_ano.withColumnRenamed("away_team", "VisitanteID")

In [194]:
partidas_2017_df_placar = partidas_2017_df_ano.select(f.split(partidas_2017_df_ano.score, 'x')).rdd.flatMap(lambda x: x).toDF(schema=['PlacarCasa','PlacarVisitante'])

AttributeError: 'DataFrame' object has no attribute 'score'

In [180]:
partidas_2017_df_placar.show(360)

+----------+---------------+
|PlacarCasa|PlacarVisitante|
+----------+---------------+
|        3 |              2|
|        1 |              1|
|        4 |              0|
|        1 |              1|
|        1 |              0|
|        4 |              1|
|        2 |              0|
|        6 |              2|
|        4 |              0|
|        0 |              0|
|        2 |              0|
|        2 |              1|
|        1 |              0|
|        2 |              0|
|        1 |              2|
|        0 |              2|
|        1 |              1|
|        0 |              1|
|        0 |              3|
|        1 |              0|
|        1 |              0|
|        3 |              2|
|        0 |              1|
|        2 |              0|
|        2 |              2|
|        1 |              1|
|        4 |              3|
|        0 |              1|
|        0 |              1|
|        2 |              0|
|        2 |              1|
|        0 |  

In [181]:
partidas_2017_df_ano = partidas_2017_df_ano(partidas_2017_df_placar
partidas_2017_df_ano.toPandas()

,PlacarCasa,PlacarVisitante
0,3,2
1,1,1
2,4,0
3,1,1
4,1,0
...,...,...
375,3,0
376,1,0
377,1,2
378,1,1


In [177]:
PlacarCasa = partidas_2017_df_placar.str.get(0)
PlacarVisitante = partidas_2017_df_placar.str.get(1)

AttributeError: 'DataFrame' object has no attribute 'str'

In [165]:
partidas_2017_df_ano = partidas_2017_df_ano.withColumn("ID", partidas_2017_df_ano["ID"].cast(IntegerType()))
partidas_2017_df_ano = partidas_2017_df_ano.withColumn("Rodada", partidas_2017_df_ano["Rodada"].cast(IntegerType()))

AnalysisException: 'Cannot resolve column name "ID" among (PlacarCasa, PlacarVisitante);'

In [156]:
partidas_2017_df_ano.toPandas()

,ID,Rodada,CasaID,score,VisitanteID,ano
0,1,1,Fluminense - RJ,3 x 2,Santos - SP,2017
1,2,1,Flamengo - RJ,1 x 1,Atlético - MG,2017
2,3,1,Palmeiras - SP,4 x 0,Vasco da Gama - RJ,2017
3,4,1,Corinthians - SP,1 x 1,Chapecoense - SC,2017
4,5,1,Cruzeiro - MG,1 x 0,São Paulo - SP,2017
...,...,...,...,...,...,...
375,376,38,Atlético - PR,3 x 0,Palmeiras - SP,2017
376,377,38,Sport - PE,1 x 0,Corinthians - SP,2017
377,378,38,Vitória - BA,1 x 2,Flamengo - RJ,2017
378,379,38,Atlético - GO,1 x 1,Fluminense - RJ,2017


In [141]:
#  ID|Rodada|CasaID|VisitanteID|PlacarCasa|PlacarVisitante|Resultado| ano
partidas_2017_df_col = partidas_2017_df_ano(usecols=[0,1,3,4,5])
partidas_2017_df_col.head()

TypeError: 'DataFrame' object is not callable

In [ ]:
#SEPARAR SCORE

In [ ]:
partidas_2017_df_ano.write.parquet('/cartola/clean/partidas_ids/2017')

In [88]:
partidas_ids_2016_parquet.show()

print(partidas_ids_2016_parquet.count())

AttributeError: 'list' object has no attribute 'game'

In [86]:
partidas_2017_df_ano.toPandas()

AttributeError: 'list' object has no attribute 'toPandas'

In [85]:
partidas_2017_df_ano.write.parquet('/cartola/clean/partidas_ids/2017')

AttributeError: 'list' object has no attribute 'write'

In [60]:
partidas_2017_parquet = spark.read.parquet('/cartola/clean/partidas_ids/*')

In [19]:
partidas_2017_parquet.show()
partidas_2017_parquet.schema

+----+------+----+---------+----------+---------------+---------+----+
|  ID|Rodada|Casa|Visitante|PlacarCasa|PlacarVisitante|Resultado| ano|
+----+------+----+---------+----------+---------------+---------+----+
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|  null|null|     null|      null|           null|     null|2017|
|null|

'0.25.3'

In [18]:
partidas_2017_parquet = partidas_2017_parquet.withColumn("ID", partidas_2017_parquet["ID"].cast(IntegerType()))
partidas_2017_parquet = partidas_2017_parquet.withColumn("Rodada", partidas_2017_parquet["rouRodadand"].cast(IntegerType()))
partidas_2017_parquet = partidas_2017_parquet.withColumn("Visitante", partidas_2017_parquet["Visitante"].cast(IntegerType()))
partidas_2017_parquet = partidas_2017_parquet.withColumn("PlacarCasa", partidas_2017_parquet["PlacarCasa"].cast(IntegerType()))
partidas_2017_parquet = partidas_2017_parquet.withColumn("PlacarVisitante", partidas_2017_parquet["PlacarVisitante"].cast(IntegerType()))

AnalysisException: 'Cannot resolve column name "rouRodadand" among (ID, Rodada, Casa, Visitante, PlacarCasa, PlacarVisitante, Resultado, ano);'

In [9]:
partidas_2017_parquet.show()

print(partidas_2017_parquet.count())

+---+----+-----+------------------+------------------+-----+------------------+--------------------+----+----+
|_c0|game|round|              date|         home_team|score|         away_team|               arena|   X| ano|
+---+----+-----+------------------+------------------+-----+------------------+--------------------+----+----+
|  1|   1|    1|14/05/2017 - 11:00|   Fluminense - RJ|3 x 2|       Santos - SP|Maracanã - Rio de...|null|2017|
|  2|   2|    1|13/05/2017 - 16:00|     Flamengo - RJ|1 x 1|     Atlético - MG|Maracanã - Rio de...|null|2017|
|  3|   3|    1|14/05/2017 - 16:00|    Palmeiras - SP|4 x 0|Vasco da Gama - RJ|Allianz Parque - ...|null|2017|
|  4|   4|    1|13/05/2017 - 19:00|  Corinthians - SP|1 x 1|  Chapecoense - SC|Arena Corinthians...|null|2017|
|  5|   5|    1|14/05/2017 - 16:00|     Cruzeiro - MG|1 x 0|    São Paulo - SP|Mineirão - Belo H...|null|2017|
|  6|   6|    1|15/05/2017 - 20:00|     Coritiba - PR|4 x 1|     Atlético - GO|Couto Pereira - C...|null|2017|
|

## scouts_raw (Analisar scouts)

In [233]:
scouts_raw_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_scouts_raw.csv", header=True)

In [234]:
scouts_raw_2017_ano = scouts_raw_2017_df.withColumn('ano', lit(2017))

In [235]:
scouts_raw_2017_ano.show()

+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----------------------+---------------+-----------------+--------------------------+----------------+--------------------+-----------------+-----------------+--------------------+------------------+------------------+-----------------+------+-----------------+--------------------+----+
|_c0|   A|  CA|  CV|  DD|  DP|  FC|  FD|  FF|  FS|  FT|   G|  GC|  GS|   I|  PE|  PP|  RB|  SG|athletes.atletas.scout|atletas.apelido|atletas.atleta_id|atletas.clube.id.full.name|atletas.clube_id|        atletas.foto|atletas.jogos_num|atletas.media_num|        atletas.nome|atletas.pontos_num|atletas.posicao_id|atletas.preco_num|Rodada|atletas.status_id|atletas.variacao_num| ano|
+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----------------------+---------------+-----------------+--------------------------+----------------+--------------------+-----------------+-

In [236]:
# Removendo colunas |_c0|   A|  CA|  CV|  DD|  DP|  FC|  FD|  FF|  FS|  FT|   G|  GC|  GS|   I|  PE|  PP|  RB|  SG|athletes.atletas.scout|atletas.apelido|atletas.atleta_id|atletas.clube.id.full.name|atletas.clube_id|        atletas.foto|atletas.jogos_num|atletas.media_num|        atletas.nome|atletas.pontos_num|atletas.posicao_id|atletas.preco_num|Rodada|atletas.status_id|atletas.variacao_num| ano|
scouts_raw_2017_ano = scouts_raw_2017_ano.drop('_c0', 'FC', 'A', 'GC', 'CA', 'CV', 'DD', 'DP', 'FD', 'FF', 'FS', 'FT', 'FT', 'G', 'GS', 'I', 'PE', 'PP', 'RB', 'SG', 'atletas.foto', 'atletas.jogos_num', 'atletas.variacao_num', 'atletas.apelido','atletas.clube.id.full.name', 'atletas.nome', 'atletas.posicao_id', 'atletas.status_id', 'atletas.variacao_num', 'athletes.atletas.scout' )

In [237]:
scouts_raw_2017_ano.limit(5).toPandas()

,atletas.atleta_id,atletas.clube_id,atletas.media_num,atletas.pontos_num,atletas.preco_num,Rodada,ano
0,36540,FLA,0,0,5,0,2017
1,36612,PAL,0,0,8,0,2017
2,36943,ATL,0,0,10,0,2017
3,37245,BAH,0,0,4,0,2017
4,37246,SPO,0,0,4,0,2017


In [238]:
#Rodada	ClubeID	AtletaID	Pontos	PontosMedia	Preco	ano
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed("athletes.atletas.scout", "Pontos")
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed("atletas.atleta_id", "AtletaID")
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed("atletas.clube_id", "ClubeID")
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed("atletas.media_num", "PontosMedia")
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed("atletas.pontos_num", "Pontos")
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed("atletas.preco_num", "Preco")

In [239]:
# remove todas as linhas com rodada == 0
scouts_raw_2017_ano_clean = scouts_raw_2017_ano[scouts_raw_2017_ano['Rodada'] > 0]

In [240]:
scouts_raw_2017_ano_clean.limit(50).toPandas()

,AtletaID,ClubeID,PontosMedia,Pontos,Preco,Rodada,ano
0,36540,FLA,0.0,0.0,5.0,1,2017
1,36612,PAL,5.1,5.1,9.27,1,2017
2,36943,ATL,1.64,1.64,8.23,1,2017
3,37245,BAH,8.39,8.39,8.76,1,2017
4,37246,SPO,-0.77,-0.77,2.28,1,2017
5,37281,CRU,5.4,5.4,12.31,1,2017
6,37306,PON,9.75,9.75,9.72,1,2017
7,37333,ATL,2.6,2.6,3.25,1,2017
8,37604,ATL,0.0,0.0,6.0,1,2017
9,37607,PAL,0.0,0.0,8.0,1,2017


In [241]:
scouts_raw_2017_ano_clean.write.parquet('/cartola/clean/scouts_raw/2017')

In [242]:
scouts_raw_2017_parquet = spark.read.parquet('/cartola/clean/scouts_raw/*')

In [246]:
scouts_raw_2017_parquet.schema
scouts_raw_2017_parquet.limit(10).toPandas()

,AtletaID,Rodada,ClubeID,Pontos,PontosMedia,Preco,ano
0,36540,1,None,0.0,0.0,5.0,2017
1,36612,1,None,5.1,5.1,9.27,2017
2,36943,1,None,1.64,1.64,8.23,2017
3,37245,1,None,8.39,8.39,8.76,2017
4,37246,1,None,-0.77,-0.77,2.28,2017
5,37281,1,None,5.4,5.4,12.31,2017
6,37306,1,None,9.75,9.75,9.72,2017
7,37333,1,None,2.6,2.6,3.25,2017
8,37604,1,None,0.0,0.0,6.0,2017
9,37607,1,None,0.0,0.0,8.0,2017


In [244]:
#deve-se converter valores em BigDecimal?
scouts_raw_2017_parquet = scouts_raw_2017_parquet.withColumn("Rodada", scouts_raw_2017_parquet["Rodada"].cast(IntegerType()))
scouts_raw_2017_parquet = scouts_raw_2017_parquet.withColumn("ClubeID", scouts_raw_2017_parquet["ClubeID"].cast(IntegerType()))
scouts_raw_2017_parquet = scouts_raw_2017_parquet.withColumn("AtletaID", scouts_raw_2017_parquet["AtletaID"].cast(IntegerType()))

In [245]:
scouts_raw_2017_parquet.show()

print(scouts_raw_2017_parquet.count())

+--------+------+-------+------+-----------+-----+----+
|AtletaID|Rodada|ClubeID|Pontos|PontosMedia|Preco| ano|
+--------+------+-------+------+-----------+-----+----+
|   36540|     1|   null|   0.0|        0.0|  5.0|2017|
|   36612|     1|   null|   5.1|        5.1| 9.27|2017|
|   36943|     1|   null|  1.64|       1.64| 8.23|2017|
|   37245|     1|   null|  8.39|       8.39| 8.76|2017|
|   37246|     1|   null| -0.77|      -0.77| 2.28|2017|
|   37281|     1|   null|   5.4|        5.4|12.31|2017|
|   37306|     1|   null|  9.75|       9.75| 9.72|2017|
|   37333|     1|   null|   2.6|        2.6| 3.25|2017|
|   37604|     1|   null|   0.0|        0.0|  6.0|2017|
|   37607|     1|   null|   0.0|        0.0|  8.0|2017|
|   37608|     1|   null|   0.0|        0.0|  4.0|2017|
|   37623|     1|   null|  -1.2|       -1.2|  4.1|2017|
|   37627|     1|   null|   0.0|        0.0|  3.0|2017|
|   37644|     1|   null|   0.0|        0.0| 10.0|2017|
|   37645|     1|   null|   0.0|        0.0|  4.

## Tabela

In [126]:
tabela_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_tabela.csv", header=True)

In [127]:
sorted_tabela_2017_df = tabela_2017_df.sort(tabela_2017_df.Pos.desc())

In [128]:
tabela_2017_ano = sorted_tabela_2017_df.withColumn('ano', lit(2017))
tabela_2017_ano.show()

+---+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|Pos|             Clube|  P|  J|  V|  E|  D| GP| GC| SG| VM| VV| DM| DV| CA| CV|  %| ano|
+---+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
| 9º|     Atlético - MG| 54| 38| 14| 12| 12| 52| 49|  3|  7|  7|  8|  4|105|  3| 47|2017|
| 8º|  Chapecoense - SC| 54| 38| 15|  9| 14| 47| 49| -2|  9|  6|  7|  7|105|  2| 47|2017|
| 7º|Vasco da Gama - RJ| 56| 38| 15| 11| 12| 40| 47| -7|  8|  7|  4|  8|105|  2| 49|2017|
| 6º|     Flamengo - RJ| 56| 38| 15| 11| 12| 49| 38| 11| 10|  5|  3|  9| 74|  2| 49|2017|
| 5º|     Cruzeiro - MG| 57| 38| 15| 12| 11| 47| 39|  8|  9|  6|  3|  8| 87|  5| 50|2017|
| 4º|       Grêmio - RS| 62| 38| 18|  8| 12| 55| 36| 19| 10|  8|  5|  7| 69|  2| 54|2017|
| 3º|       Santos - SP| 63| 38| 17| 12|  9| 42| 32| 10| 12|  5|  3|  6| 94|  4| 55|2017|
| 2º|    Palmeiras - SP| 63| 38| 19|  6| 13| 61| 45| 16| 12|  7|  4|  9| 87|  4| 55|2017|
|20º|     

In [ ]:
tabela_2017_ano.write.partitionBy('ano').parquet('/cartola/2017/clean/tabela')

In [ ]:
times_2017_parquet = spark.read.parquet("/cartola/2017/clean/tabela")

In [ ]:
times_2017_parquet.show()
times_2017_parquet.schema

#Alterar ID para int.(está string)

In [ ]:
times_2017_parquet = times_2017_parquet.withColumn("P", times_2017_parquet["P"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("J", times_2017_parquet["J"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("V", times_2017_parquet["V"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("E", times_2017_parquet["E"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("D", times_2017_parquet["D"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("GP", times_2017_parquet["GP"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("GC", times_2017_parquet["GC"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("SG", times_2017_parquet["SG"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("VM", times_2017_parquet["VM"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("VV", times_2017_parquet["VV"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("DM", times_2017_parquet["DM"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("DV", times_2017_parquet["DV"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("CA", times_2017_parquet["CA"].cast(IntegerType()))
times_2017_parquet = times_2017_parquet.withColumn("CV", times_2017_parquet["CV"].cast(IntegerType()))

In [ ]:
times_2017_parquet.schema

In [ ]:
times_2017_parquet.show()

print(times_2017_parquet.count())

## Times

In [247]:
#Cabeçalho separado
times_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_times.csv", header=True, sep=';')

In [249]:
sorted_times_2017_df = times_2017_df.sort(times_2017_df.ID.desc())
times_2017_df.show()

+---+-----------+----------+------------------+
| ID|       Nome|Abreviacao|              Slug|
+---+-----------+----------+------------------+
|373|Atlético-GO|       ATL|     Atlético - GO|
|282|Atlético-MG|       ATL|     Atlético - MG|
|293|Atlético-PR|       ATL|     Atlético - PR|
|314|       Avaí|       AVA|         Avaí - SC|
|265|      Bahia|       BAH|        Bahia - BA|
|263|   Botafogo|       BOT|     Botafogo - RJ|
|315|Chapecoense|       CHA|  Chapecoense - SC|
|264|Corinthians|       COR|  Corinthians - SP|
|294|   Coritiba|       COR|     Coritiba - PR|
|283|   Cruzeiro|       CRU|     Cruzeiro - MG|
|262|   Flamengo|       FLA|     Flamengo - RJ|
|266| Fluminense|       FLU|   Fluminense - RJ|
|284|     Grêmio|       GRE|       Grêmio - RS|
|275|  Palmeiras|       PAL|    Palmeiras - SP|
|303|Ponte Preta|       PON|  Ponte Preta - SP|
|277|     Santos|       SAN|       Santos - SP|
|276|  São Paulo|       SAO|    São Paulo - SP|
|292|      Sport|       SPO|        Spor

In [250]:
sorted_times_2017_df = times_2017_df.sort(times_2017_df.ID.desc())

In [251]:
times_2017_ano = sorted_times_2017_df.withColumn('ano', lit(2017))
times_2017_ano.show()

+---+-----------+----------+------------------+----+
| ID|       Nome|Abreviacao|              Slug| ano|
+---+-----------+----------+------------------+----+
|373|Atlético-GO|       ATL|     Atlético - GO|2017|
|315|Chapecoense|       CHA|  Chapecoense - SC|2017|
|314|       Avaí|       AVA|         Avaí - SC|2017|
|303|Ponte Preta|       PON|  Ponte Preta - SP|2017|
|294|   Coritiba|       COR|     Coritiba - PR|2017|
|293|Atlético-PR|       ATL|     Atlético - PR|2017|
|292|      Sport|       SPO|        Sport - PE|2017|
|287|    Vitória|       VIT|      Vitória - BA|2017|
|284|     Grêmio|       GRE|       Grêmio - RS|2017|
|283|   Cruzeiro|       CRU|     Cruzeiro - MG|2017|
|282|Atlético-MG|       ATL|     Atlético - MG|2017|
|277|     Santos|       SAN|       Santos - SP|2017|
|276|  São Paulo|       SAO|    São Paulo - SP|2017|
|275|  Palmeiras|       PAL|    Palmeiras - SP|2017|
|267|      Vasco|       VAS|Vasco da Gama - RJ|2017|
|266| Fluminense|       FLU|   Fluminense - RJ

In [252]:
sorted_times_2017_df.write.parquet('/cartola/clean/times/2017/')

AnalysisException: 'path hdfs://namenode:8020/cartola/clean/times/2017 already exists.;'

In [136]:
times_2017_parquet = spark.read.parquet('/cartola/clean/times/*')

In [137]:
times_2017_parquet.show()
times_2017_parquet.schema

#Alterar ID para int.(está string)

+---+-------------+----------+-------------+----+
| ID|         Nome|Abreviacao|         Slug| ano|
+---+-------------+----------+-------------+----+
|262|     flamengo|       FLA|     flamengo|2014|
|263|     botafogo|       BOT|     botafogo|2014|
|264|  corinthians|       COR|  corinthians|2014|
|265|        bahia|       BAH|        bahia|2014|
|266|   fluminense|       FLU|   fluminense|2014|
|275|    palmeiras|       PAL|    palmeiras|2014|
|276|    são paulo|       SAO|    sao-paulo|2014|
|277|       santos|       SAN|       santos|2014|
|282|  atlético-mg|       CAM|  atletico-mg|2014|
|283|     cruzeiro|       CRU|     cruzeiro|2014|
|284|       grêmio|       GRE|       gremio|2014|
|285|internacional|       INT|internacional|2014|
|287|      vitória|       VIT|      vitoria|2014|
|288|     criciúma|       CRI|     criciuma|2014|
|290|        goiás|       GOI|        goias|2014|
|292|        sport|       SPO|        sport|2014|
|293|  atlético-pr|       CAP|  atletico-pr|2014|


StructType(List(StructField(ID,StringType,true),StructField(Nome,StringType,true),StructField(Abreviacao,StringType,true),StructField(Slug,StringType,true),StructField(ano,IntegerType,true)))

In [138]:
times_2017_parquet = times_2017_parquet.withColumn("ID", times_2017_parquet["ID"].cast(IntegerType()))

In [139]:
times_2017_parquet.schema

StructType(List(StructField(ID,IntegerType,true),StructField(Nome,StringType,true),StructField(Abreviacao,StringType,true),StructField(Slug,StringType,true),StructField(ano,IntegerType,true)))

In [140]:
times_2017_parquet.show()

print(times_2017_parquet.count())

+---+-------------+----------+-------------+----+
| ID|         Nome|Abreviacao|         Slug| ano|
+---+-------------+----------+-------------+----+
|262|     flamengo|       FLA|     flamengo|2014|
|263|     botafogo|       BOT|     botafogo|2014|
|264|  corinthians|       COR|  corinthians|2014|
|265|        bahia|       BAH|        bahia|2014|
|266|   fluminense|       FLU|   fluminense|2014|
|275|    palmeiras|       PAL|    palmeiras|2014|
|276|    são paulo|       SAO|    sao-paulo|2014|
|277|       santos|       SAN|       santos|2014|
|282|  atlético-mg|       CAM|  atletico-mg|2014|
|283|     cruzeiro|       CRU|     cruzeiro|2014|
|284|       grêmio|       GRE|       gremio|2014|
|285|internacional|       INT|internacional|2014|
|287|      vitória|       VIT|      vitoria|2014|
|288|     criciúma|       CRI|     criciuma|2014|
|290|        goiás|       GOI|        goias|2014|
|292|        sport|       SPO|        sport|2014|
|293|  atlético-pr|       CAP|  atletico-pr|2014|


## DEMONSTRAÇÃO

In [ ]:
partidas_2014_df = spark.read.csv("/cartola/data/2014/2014_jogadores.csv", header=True)
partidas_2014_ct = partidas_2014_df.withColumn('time', regexp_replace('home_team', ' - RJ', ''))
final_partidas = partidas_2014_ct.withColumn('time_low', lower(col('time'))).show(truncate=False)

In [ ]:
with_ano_partidas = partidas_2014_ct.withColumn('ano', lit(2014)).show(truncate=False)
with_ano_partidas.show()